In [195]:
class Cycler:
    def __init__(self, values, counter: int =0):
        self.values = list(values)
        self.counter = counter

    def __iter__(self):
        return self.values.__iter__()

    def next(self):
        idx = self.counter % len(self.values)
        self.counter += 1
        return self.values[idx] 

    def copy(self, reset: bool = True):
        counter = 0 if reset else self.counter
        return self.__class__(self.values, counter)

    def __repr__(self):
        cname = self.__class__.__name__
        return f"{cname}({self.values!r}, {self.counter})"


cycler = Cycler(["red", "green", "blue"])
cycler


Cycler(['red', 'green', 'blue'], 0)

In [196]:

class Stylesheet:
    def __init__(self, stylemap):
        if hasattr(stylemap, 'stylemap'):
            stylemap = stylemap.stylemap

        def map_value(value):
            if isinstance(value, (list, Cycler)):
                return Cycler(value)
            return value
        self.stylemap = {
            k: map_value(v) for k, v in stylemap.items()
        }


    def __repr__(self):
        cname = self.__class__.__name__
        return f"{cname}({self.stylemap!r})"

    def _repr_pretty_(self, p, cycle):
        cname = self.__class__.__name__
        if cycle:
            p.text(f"{cname}(...)")
        else:
            with p.group(8, f"{cname}(", ")"):
                p.pretty(self.stylemap)

    def get_setting(self, *keys, facet: str = None):
        if facet is not None:
            keys = tuple(k+":"+facet for k in keys)
        for key in keys:
            value = self.stylemap.get(key)
            if value is None:
                continue
            if isinstance(value, Cycler):
                value = value.next()
            return value   


    def copy(self):
        return self.__class__(self.stylemap)


stylemap = {
    "candlesticks.up:color": "black",
    "candlesticks.down:color": "red",
    "sma:color" : ["blue", "green", "red"]
}


stylesheet = Stylesheet(stylemap)

print(stylesheet)
display(stylesheet)

Stylesheet({'candlesticks.up:color': 'black', 'candlesticks.down:color': 'red', 'sma:color': Cycler(['blue', 'green', 'red'], 0)})


Stylesheet({'candlesticks.up:color': 'black',
         'candlesticks.down:color': 'red',
         'sma:color': Cycler(['blue', 'green', 'red'], 0)})

In [197]:
print(stylesheet.get_setting("sma", facet="color"))
print(stylesheet.get_setting("sma", facet="color"))
stylesheet


blue
green


Stylesheet({'candlesticks.up:color': 'black',
         'candlesticks.down:color': 'red',
         'sma:color': Cycler(['blue', 'green', 'red'], 2)})

In [198]:
stylesheet.copy()

Stylesheet({'candlesticks.up:color': 'black',
         'candlesticks.down:color': 'red',
         'sma:color': Cycler(['blue', 'green', 'red'], 0)})

In [199]:
from collections import defaultdict

class Chart:

    def __init__(self, stylemap):
        self.stylesheet = Stylesheet(stylemap)
        self.stylecache = defaultdict(lambda: Stylesheet(self.stylesheet))

    def get_stylesheet(self, ax):
        return self.stylecache[ax]


chart = Chart(stylemap=stylesheet)
chart.get_stylesheet(1)


Stylesheet({'candlesticks.up:color': 'black',
         'candlesticks.down:color': 'red',
         'sma:color': Cycler(['blue', 'green', 'red'], 0)})

In [200]:
class MyDict(dict):
    def __init__(self, mapping=(), **kwargs):
        super().__init__(mapping, **kwargs)
    
    def __repr__(self):
        cname = self.__class__.__name__
        return f"{cname}({super().__repr__()})"


MyDict(a=1, b=2)

MyDict({'a': 1, 'b': 2})